In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import sys
from importlib import reload


from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LinearRegression
from sklearn.manifold import TSNE
import torch as t
from sklearn.metrics import classification_report

# sys.path.append("~/Documents/Prog/LLMs/mri_llm")
sys.path.append("..") # Adds higher directory to python modules path.

from datas import celebrity, conspiracy, english_word, character

In [2]:
files_root = "../inference_data/"
files = [
    # "inference_conspiracy_character_sentence_908.pkl",
    # "inference_conspiracy_711.pkl",
    # "inference_character_474.pkl",
    # "inference_celebrity_random_764.pkl",
    # "inference_celebrity_285.pkl",
    # "inference_english_word_236.pkl",
    # "inference_date_884.pkl",
    # "inference_election_195.pkl",
    # "inference_date_election_648.pkl",

    "inference_conspiracy_horizontal_593.pkl",
    "inference_character_horizontal_945.pkl",

    # "inference_celebrity_horizontal_604.pkl",
    # "inference_english_word_horizontal_252.pkl",
]

data_list=[]
for f in files:
    with open(files_root+f, "rb") as fp:
        data_list.extend(pickle.load(fp))

In [3]:
print(len(data_list))
print(data_list[0].system_prompt)
print(data_list[0].user_prompt)
print(data_list[0].input_text)
print(data_list[0].output_text)

149
Always respond with a SINGLE sentence. You are given a conspiracy name, give me a short description.
{}:
Watergate Scandal
The Watergate Scandal was a political scandal that erupted in 1972 when it was discovered that members of the Nixon administration were involved in the break-in and subsequent cover-up of the Democratic National Committee headquarters at the Watergate complex in Washington, D.C.


In [4]:
def type_of_input_labeling(data_elmt):
    label = data_elmt.label
    match data_elmt.__class__.__name__:
        # HALLUCINATION STUFF
        case "celebrity":
            label += 0
        case "conspiracy":
            label += 2*2
        case "english_word":
            label += 2*1
        # TIME STUFF
        case "date":
            label += 0
        case "election": # Carefull this guy has 3 switches
            label += 2*1
        # FICTION STUFF
        case "character":
            label += 0
        case _ :
            raise Exception("Error of non recognised type of data element")

    if data_elmt.sufix and data_elmt.sufix== "random":
        label = data_elmt.label + 2
        
    return label

In [5]:
X = np.array([[data_list[i].activations[lay_nb][0].type(t.float64).cpu().numpy() for lay_nb in range(len(data_list[i].activations))] for i in range(len(data_list))])
Y = np.array([type_of_input_labeling(data_list[i]) for i in range(len(data_list))])

# torch.stack().type(torch.float64).cpu().numpy()
print(len(X))
print(len(X[0]))
print(len(X[0][0]))
print(X[0][0])
print(Y)

from collections import Counter
print(Counter(Y))

149
1
4096
[-0.01452637  0.08496094 -0.02819824 ...  0.07421875  0.12207031
 -0.02258301]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
Counter({5: 57, 4: 52, 1: 22, 0: 18})


In [6]:
color_selction = ["#ef6c00", "#eeff41", "#009668", "#ce93d8" , "#a1e8d9", "#b3a77d", "#6a634a", "red", "green"]

def simple_label2color(label):
    # labels are 4 choices
    color = color_selction[label]
    return color

def bidim_label2color(label):
    # labels are 2*2*2 choices
    color = color_selction[label[1] + 2*label[2]]
    # color = mcolors.CSS4_COLORS[color_selction[label[1] + 2*label[2]]]
    return color

## Add some labels according to other conditions

- Length of output

In [7]:
def output_length_label(data_elmt):
    return len(data_elmt.output_text)
    # if len(data_elmt.output_text) > 100:
    #     return "long_output"
    # else:
    #     return "short_output"

for i in range(len(data_list)):
    data_list[i].output_length = output_length_label(data_list[i])
    data_list[i].class_type = data_list[i].__class__.__name__


In [8]:
def normlise_by_data_type(data_list, states):
    type_indices = {}
    for i in range(len(data_list)):
        # Select according to certain parameter, for exemple class type of data
        data_type = data_list[i].__class__.__name__
        if data_type in type_indices:
            type_indices[data_type].append(i)
        else:
            type_indices[data_type] = [i]
    for key in type_indices:
        indices = np.array(type_indices[key])
        states[indices] = states[indices] - states[indices].mean(axis=0)

    return states

In [9]:
# TSNE plot stuff
def plot_tsne(X, Y):
    inp = np.array([x for x in X])
    y = np.array([yy for yy in Y]) # list(range(len(data_el.activations[0])))

    tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
    res = tsne.fit_transform(inp)

    plt.scatter(res[:, 0], res[:, 1], c=y)
    plt.show()

In [144]:
def tensor2array(te):
    return te.type(t.float64).cpu().numpy()

def activation_selection(data_el):

    if data_el.sufix == "horizontal":
        # For the only layer there is, we divide activation between prompt and generated text
        end_of_prompt_id = data_el.input_token_length-1     # add -1 because during process of last prompt token we are alreaddy generating new content.
        prompt_activations = data_el.activations[0][0:end_of_prompt_id]
        generated_activations = data_el.activations[0][end_of_prompt_id:]

    # test 1: Sum(gen_act) - Sum(promp_act)
    # result = t.stack(generated_activations).sum(dim=0) - t.stack(prompt_activations).sum(dim=0)
    # # Seems to create even more difference between datasets

    # # test 2: the baseline just last line
    # result = generated_activations[-1]

    # # test 3: add generated tokens
    # result = t.stack(generated_activations).sum(dim=0)

    # test 4: add all tokens
    # result = t.stack(prompt_activations + generated_activations).sum(dim=0)

    # test 5: concatenate last 2 gen tokens
    # result = t.cat(generated_activations[-2:])

    # # test 6: ultipenultime gen tokens
    # result = generated_activations[-2]

    # # test 7: mean of each row of gen tokens
    # result = t.stack(generated_activations).mean(dim=0)

    # # test 8: mean of each row of gen tokens sub promp mean
    # result = t.stack(generated_activations).mean(dim=0) - t.stack(prompt_activations).mean(dim=0)

    # # test 9: Clipped to threshold + mean gen
    result = t.clip(t.stack(generated_activations), min=5, max=None)
    result[result == 5] = 0
    print(result[0])
    result = result.mean(dim=0)
    # result = generated_activations[-1]

    # print(result.shape)

    return  tensor2array(result)


out = activation_selection(data_list[0])
print(out)
print(len(np.where(out>0)[0]))

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
[0.         0.         0.         ... 0.10302734 0.         0.        ]
1019


In [145]:
def train_and_fit_flat(train_condition, mode="PCA", group_norm=False):
    X = np.array([activation_selection(data_list[i]) for i in range(len(data_list))])
    Y = np.array([type_of_input_labeling(data_list[i]) for i in range(len(data_list))])

    # Do a PCA projection for each layer
    layer_pca = []
    pca_training_data_indices = np.array(list(filter(train_condition, enumerate(data_list))))[:, 0]
    print("Size of training data:", len(pca_training_data_indices))

    # Train on one problem
    data_states = X
    if group_norm:
        data_states = normlise_by_data_type(data_list, data_states)
    if mode=="PCA":
        pca = PCA(n_components=4)
        train_d = [data_states[i] for i in pca_training_data_indices]
        pca.fit(train_d)
        train_d -= np.mean(train_d, axis=0)
        cov_matrix = np.dot(train_d.T, train_d) / len(train_d)
        for eigenvector in pca.components_:
            print("Eigenvalue: ", np.dot(eigenvector.T, np.dot(cov_matrix, eigenvector)))
    elif mode=="LDA":
        pca = LDA(n_components=1)    
        pca.fit([data_states[i] for i in pca_training_data_indices], [Y[i] for i in pca_training_data_indices])
    x = pca.transform(data_states)

    # Every label is multiplied by the number of layers in order to have a data point for each input x layer
    # Labels
    input_text = np.array([data_list[i].input_text  for i in range(len(data_list))])
    data_type = np.array([data_list[i].__class__.__name__  for i in range(len(data_list))])
    output_length = np.array([data_list[i].output_length  for i in range(len(data_list))]) 
    swtich = np.array([data_list[i].switch_name  for i in range(len(data_list))])
    output_text = np.array([data_list[i].output_text  for i in range(len(data_list))])
    sufix = np.array([1 if data_list[i].sufix else 0  for i in range(len(data_list))])

    if mode=="PCA":
        df = pd.DataFrame(zip(x[:, 0], x[:, 1], x[:, 2], x[:, 3], input_text, Y, data_type, output_length, swtich, output_text, sufix), 
                    columns=['pca1', 'pca2', 'pca3', 'pca4', "input/output", "label", "data_type", "output_length", "switch", "output_text", "sufix"])
    elif mode=="LDA":
        df = pd.DataFrame(zip(x[:, 0], input_text, Y, data_type, output_length, swtich, output_text, sufix), 
                    columns=['pca1', "input/output", "label", "data_type", "output_length", "switch", "output_text", "sufix"])
    return df

In [146]:
def train_and_fit_pca(train_condition, mode="PCA"):
    X = np.array([activation_selection(data_list[i]) for i in range(len(data_list))])
    Y = np.array([type_of_input_labeling(data_list[i]) for i in range(len(data_list))])

    label_colors = [simple_label2color(y) for y in Y]

    # Do a PCA projection for each layer
    layer_pca = []
    layer_id = []
    pca_training_data_indices = np.array(list(filter(train_condition, enumerate(data_list))))[:, 0]
    print("Size of training data:", len(pca_training_data_indices))
    for layer_nb in range(32):
        # Train on one problem
        data_states = X[:, layer_nb]
        data_states = normlise_by_data_type(data_list, data_states)
        if mode=="PCA":
            pca = PCA(n_components=2)    
            train_d = [data_states[i] for i in pca_training_data_indices]
            pca.fit(train_d)
            train_d -= np.mean(train_d, axis=0)
            cov_matrix = np.dot(train_d.T, train_d) / len(train_d)
            for eigenvector in pca.components_:
                print("Eigenvalue: ", np.dot(eigenvector.T, np.dot(cov_matrix, eigenvector)))
        elif mode=="LDA":
            pca = LDA(n_components=1)    
            pca.fit([data_states[i] for i in pca_training_data_indices], [Y[i] for i in pca_training_data_indices])
        x = pca.transform(data_states)
        layer_pca.extend(x)
        a = np.empty(len(x))
        a.fill(layer_nb)
        layer_id.extend(a)

    layer_pca = np.array(layer_pca)
    layer_id = np.array(layer_id)

    # Every label is multiplied by the number of layers in order to have a data point for each input x layer
    # Labels
    # y = np.array([data_list[i].label for i in range(len(data_list))])
    y_layered = np.tile(Y, (32))

    input_text = np.array([data_list[i].input_text  for i in range(len(data_list))])
    inputs_layered = np.tile(input_text, 32)

    data_type = np.array([data_list[i].__class__.__name__  for i in range(len(data_list))])
    data_type_layered = np.tile(data_type, 32)

    output_length = np.array([data_list[i].output_length  for i in range(len(data_list))]) 
    output_length_layered = np.tile(output_length, 32)

    swtich = np.array([data_list[i].switch_name  for i in range(len(data_list))])
    swtich_layered = np.tile(swtich, 32)

    output_text = np.array([data_list[i].output_text  for i in range(len(data_list))])
    output_text_layered = np.tile(output_text, 32)

    sufix = np.array([1 if data_list[i].sufix else 0  for i in range(len(data_list))])
    sufix_layered = np.tile(sufix, 32)

    if mode=="PCA":
        df = pd.DataFrame(zip(layer_pca[:, 0], layer_pca[:, 1], inputs_layered, layer_id, y_layered, data_type_layered, output_length_layered, swtich_layered, output_text_layered, sufix_layered), 
                    columns=['pca1', 'pca2', "input/output", "layer_id", "label", "data_type", "output_length", "switch", "output_text", "sufix"])
    elif mode=="LDA":
        df = pd.DataFrame(zip(layer_pca[:, 0], inputs_layered, layer_id, y_layered, data_type_layered, output_length_layered, swtich_layered, output_text_layered, sufix_layered), 
                    columns=['pca1', "input/output", "layer_id", "label", "data_type", "output_length", "switch", "output_text", "sufix"])

    return df

In [156]:
# TRAIN PCA

train_condition_fun = lambda el: el[1].__class__.__name__ in ["character"] #and len(el[1].output_text) < 10 #and el[1].input_text in ["Time", "Blorble", "blorple", "Teacher"]
df = train_and_fit_flat(train_condition_fun, mode="PCA", group_norm=True)

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=to

In [157]:
df[(df["data_type"] == "english_word") & (df["label"]== 2)]

,pca1,pca2,pca3,pca4,input/output,label,data_type,output_length,switch,output_text,sufix


In [158]:
# print(df[(df["data_type"] == "conspiracy") & (df["output_length"] is not None)& (df["label"] == 2) & (df["input/output"] == "Watergate Scandal") & (df["layer_id"] == 16)]["output_text"].item())

In [159]:

# df_modified = df[df["label"] ]
# df_modified = df[df["layer_id"] == 16]
# df_modified = df_modified[(df_modified["label"] < 2) | (df_modified["label"] > 3)]
# df_modified = df_modified[df_modified["sufix"] == 1]
# df_modified = df_modified[df_modified["output_length"] < 20]
# print(df_modified.head()["label"])
# print(set(df_modified[data_type]))
df_modified = df.copy()
df_modified["label"] = df_modified["label"].astype(str)
df_modified["random_value"] = np.random.rand(len(df_modified))

scatter = px.scatter(df_modified, x='pca1', y='pca2',
                     color="label",
                    #  size='output_length', 
                     symbol= "data_type",
                     hover_data=["input/output", "output_length", "switch"],
                     color_discrete_sequence=color_selction)

scatter.update_layout(
    # autosize=True,
    width=1000,
    height=700,)

scatter.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=-0.2,
                                          ticks="outside"))

scatter.show()

## Use probes for quantitative evaluation

In [160]:
import probes
reload(probes)

<module 'probes' from '/home/thibaua25/Documents/Prog/LLMs/mri_llm/notebooks/../probes.py'>

In [161]:
# split data
split_condition1 = df["data_type"]=="character"
split_condition2 = df["output_length"] < 10
train_df = df[split_condition1]
eval_df = df[~split_condition1]

# layernb = 16
# train_df = train_df[train_df["layer_id"] == layernb]
# eval_df = eval_df[eval_df["layer_id"] == layernb]


# format for training
x = train_df[['pca1', 'pca2',  'pca3', 'pca4']].to_numpy()
y = train_df['label'].to_numpy()
y = y - np.min(y)

# train probe
p = probes.MMProbe.from_data(x, y)

In [162]:
# eval data
x_eval = eval_df[['pca1', 'pca2',  'pca3', 'pca4']].to_numpy()
y_eval = eval_df['label'].to_numpy()
y_eval = y_eval - np.min(y_eval)

print(y_eval)

prediction = p.pred(x_eval)
print(prediction.numpy())

eval_df["prediction"] = prediction.numpy()

class_report = classification_report(y_eval, eval_df["prediction"].to_numpy(), output_dict=True)
print("Total cases:", len(y_eval))
print("Success cases:", class_report["accuracy"]*len(y_eval))
print("Failure cases:", (1 - class_report["accuracy"])*len(y_eval))
print("F1 score:", class_report['macro avg']['f1-score'])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Total cases: 109
Success cases: 54.0
Failure cases: 55.0
F1 score: 0.36094233024197847


/tmp/ipykernel_737287/644924848.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [163]:
loggits = p(x_eval).cpu().numpy()

confidence = 0.5

# print(np.abs(loggits - 0.5))
confident_indices = np.where(np.abs(loggits - 0.5) > (0.5 - confidence/2))[0] 
# print(confident_indices)

class_report = classification_report(y_eval[confident_indices], eval_df["prediction"].to_numpy()[confident_indices], output_dict=True)
print("Total cases:", len(y_eval), "confident cases:", len(y_eval[confident_indices]))
print("Success cases:", class_report["accuracy"]*len(y_eval[confident_indices]))
print("Failure cases:", (1 - class_report["accuracy"])*len(y_eval[confident_indices]))
print("F1 score:", class_report['macro avg']['f1-score'])

Total cases: 109 confident cases: 109
Success cases: 54.0
Failure cases: 55.0
F1 score: 0.36094233024197847


In [164]:
eval_df["random_value"] = np.random.rand(len(eval_df))
eval_df["label"] = eval_df["label"] #+ 1

scatter = px.scatter(eval_df, x='pca1', y='pca2', 
                     color="label",
                     size="output_length", 
                     symbol= "prediction",
                     hover_data=["input/output", "output_length", "switch"],
                     color_discrete_sequence=color_selction)

scatter.update_layout(
    # autosize=True,
    width=1000,
    height=700,)

scatter.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=-0.2,
                                          ticks="outside"))

scatter.add_vline(x=0, line_dash="dash", line_color="red")

scatter.show()

/tmp/ipykernel_737287/3132793002.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_737287/3132793002.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

